In [1]:
import cv2

from face_features import FaceFeatureExtractor

feature_extractor = FaceFeatureExtractor()

In [2]:
# Load image.
imgs = [
    '/home/jg/Pictures/Webcam/2018-01-14-113441.jpg',
    '/home/jg/Pictures/Webcam/2018-01-13-223825.jpg',
    '/home/jg/Pictures/Webcam/2018-01-13-173605.jpg',    
]
im = cv2.imread(imgs[1])

In [3]:
features = feature_extractor.getFeatures(im)

In [4]:
features.shape

(1, 10575)